# Choropleth Maps Exercise 

Welcome to the Choropleth Maps Exercise! In this exercise we will give you some simple datasets and ask you to create Choropleth Maps from them. Due to the Nature of Plotly we can't show you examples

[Full Documentation Reference](https://plot.ly/python/reference/#choropleth)

## Plotly Imports

In [6]:
pip install iso3166

  Obtaining dependency information for iso3166 from https://files.pythonhosted.org/packages/08/d0/bf18725b8d47f37858ff801f8e4d40c6982730a899725bdb6ded62199954/iso3166-2.1.1-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [7]:
import plotly.graph_objs as go 
import plotly.offline as pyo
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True) 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from iso3166 import countries

In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [46]:
# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_countries_by_electricity_consumption"

# Send a request to the webpage and get its content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup
# Extract the table

# table_rows = table.find_all('tr')
# # Assuming the relevant table is the first one

# res = []
# for tr in table_rows:
#     td = tr.find_all('td')
#     row = [tr.text.strip() for tr in td if tr.text.strip()]
#     if row:
#         res.append(row)
        
# res
# Display the first few rows to understand its structure


In [59]:
table = soup.find('table',attrs={'class':'sortable wikitable sticky-header-multi sort-under col2left'})
# table
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df = pd.DataFrame(res, columns=
[
    'Rank', 
    'Location', 
    'Consumption_GWh_per_year', 
    'Year', 
    'Source', 
    'Population', 
    'As_of', 
    'Consumption_per_capita_kWh_per_year', 
    'Consumption_per_capita_Watts'
])

# Drop unnecessary columns such as 'Rank', 'Source', 'Population', 'As_of'
df = df.drop(columns=['Rank', 'Source', 'Population', 'As_of'])

# Convert the 'Consumption_GWh_per_year' column to numeric, and handle commas
df['Consumption_GWh_per_year'] = df['Consumption_GWh_per_year'].str.replace(',', '').astype(float)

# Convert GWh to kWh (1 GWh = 1e6 kWh)
df['Consumption_kWh'] = df['Consumption_GWh_per_year'] * 1e6

# Keep only the relevant columns
df = df[['Location', 'Consumption_kWh', 'Year']]

# Display the cleaned DataFrame
df.head()
df.rename(columns={'Location':'Country'},inplace=True)
df

,Country,Consumption_kWh,Year
0,World,2.534300e+13,2021[4]
1,China,7.806000e+12,2021[4]
2,United States,3.979000e+12,2021[4]
3,India,1.443000e+12,2021[4]
4,Russia,9.960000e+11,2021[4]
...,...,...,...
215,Kiribati,2.775000e+07,2018 est.
216,Falkland Islands,1.832000e+07,2018 est.
217,Montserrat,1.488000e+07,2018 est.
218,Niue,7.440000e+06,2018 est.


In [76]:
from iso3166 import countries

# Function to get ISO alpha-3 code
def get_iso_alpha_3(country_name):
    try:
        return countries.get(country_name).alpha3
    except KeyError:
        return None

# Apply the function to the 'Country' column
df['ISO_alpha-3'] = df['Country'].apply(get_iso_alpha_3)

# Handle any missing or incorrect country codes manually
corrections = {
    'United States': 'USA',
    'Russia': 'RUS',
    'South Korea': 'KOR',
    'Vietnam': 'VNM',
    'Iran': 'IRN',
    'Venezuela': 'VEN',
    'Taiwan': 'TWN',
    'Syria': 'SYR'
}

df['ISO_alpha-3'] = df['Country'].replace(corrections).apply(get_iso_alpha_3)

# Display the updated DataFrame with ISO codes
df


,Country,Consumption_kWh,Year,ISO_alpha-3,ISO-3
0,World,2.534300e+13,2021[4],None,None
1,China,7.806000e+12,2021[4],CHN,CHN
2,United States,3.979000e+12,2021[4],USA,None
3,India,1.443000e+12,2021[4],IND,IND
4,Russia,9.960000e+11,2021[4],RUS,None
...,...,...,...,...,...
215,Kiribati,2.775000e+07,2018 est.,KIR,KIR
216,Falkland Islands,1.832000e+07,2018 est.,None,None
217,Montserrat,1.488000e+07,2018 est.,MSR,MSR
218,Niue,7.440000e+06,2018 est.,NIU,NIU


** Referencing the lecture notes, create a Choropleth Plot of the Power Consumption for Countries using the data and layout dictionary. **

In [68]:
# Layout for the map
layout = go.Layout(
    title='Power Consumption by Country',
    geo=dict(
        scope='world',
        projection_type='equirectangular'  # Can adjust projection type if needed
    )
)

In [77]:
# Data for the choropleth map
# Verify the alignment between country names and their data
data = go.Choropleth(
    locations=df['ISO_alpha-3'],
    locationmode='ISO-3',
    z=df['Consumption_kWh'],
    colorscale='Portland',
    text=df['Country'],
    colorbar_title="Consumption",
    marker_line_color='darkgray',  # Adds a border to the countries
    marker_line_width=0.5
)


# Constructing the figure
choromap = go.Figure(data=[data], layout=layout)

# Plotting the figure
pyo.iplot(choromap, validate=False)


## USA Choropleth

** Import the 2012_Election_Data csv file using pandas. **

In [102]:
df = pd.read_csv('2012_Election_Data')

** Check the head of the DataFrame. **

In [103]:
df.head(6)

,Year,ICPSR State Code,Alphanumeric State Code,State,VEP Total Ballots Counted,VEP Highest Office,VAP Highest Office,Total Ballots Counted,Highest Office,Voting-Eligible Population (VEP),Voting-Age Population (VAP),% Non-citizen,Prison,Probation,Parole,Total Ineligible Felon,State Abv
0,2012,41,1,Alabama,NaN,58.6%,56.0%,NaN,"2,074,338","3,539,217",3707440.0,2.6%,"32,232","57,993","8,616","71,584",AL
1,2012,81,2,Alaska,58.9%,58.7%,55.3%,"301,694","300,495","511,792",543763.0,3.8%,"5,633","7,173","1,882","11,317",AK
2,2012,61,3,Arizona,53.0%,52.6%,46.5%,"2,323,579","2,306,559","4,387,900",4959270.0,9.9%,"35,188","72,452","7,460","81,048",AZ
3,2012,42,4,Arkansas,51.1%,50.7%,47.7%,"1,078,548","1,069,468","2,109,847",2242740.0,3.5%,"14,471","30,122","23,372","53,808",AR
4,2012,71,5,California,55.7%,55.1%,45.1%,"13,202,158","13,038,547","23,681,837",28913129.0,17.4%,"119,455",0,"89,287","208,742",CA
5,2012,62,6,Colorado,70.6%,69.9%,64.5%,"2,596,173","2,569,522","3,675,871",3981208.0,6.9%,"18,807",0,"11,458","30,265",CO


** Now create a plot that displays the Voting-Age Population (VAP) per state. If you later want to play around with other columns, make sure you consider their data type. VAP has already been transformed to a float for you. **

In [104]:
# Layout for the map
layout = dict(title = '2012 Voting-Age Population (VAP) by State',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )

In [107]:
data = dict(type='choropleth',
#             colorscale = 'YIOrRd',
            locations = df['State Abv'],
            z = df['Voting-Age Population (VAP)'],
            locationmode = 'USA-states',
            text = df['State'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Millions"}
            ) 

In [108]:
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap,validate=False)

# Great Job!